In [1]:
import google.generativeai as genai
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from summarizer import extract_transcript
from youtube_id_extractor import extract_video_id
from yt_chat_processing import create_chunk_and_vector_db, use_input

In [17]:
def get_conversational_chain():

    prompt_template = """
    Answer the question as detailed as possible from the provided context, make sure to provide all the details and explain it better that it is in the context, 
    if the answer is not in provided context or even out of the scope of the context just say,
    "answer is not available in the context", don't provide the wrong answer\n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """

    model = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.3, convert_system_message_to_human=True)

    prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

    return chain



def user_input(user_question):
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001", task_type="retrieval_document")
    
    new_db = FAISS.load_local("faiss_index", embeddings)
    docs = new_db.similarity_search(user_question)

    chain = get_conversational_chain()

    
    response = chain(
        {"input_documents":docs, "question": user_question}
        , return_only_outputs=True)

    print(response)
    print("Reply: ", response["output_text"])

In [18]:
user_question = 'what are the four steps that can be used to build llm from scratch?'

user_input(user_question)

{'output_text': 'The four steps that can be used to build LLM from scratch are:\n\n1. Data Curation: This involves gathering and preparing a large and diverse dataset of text, which is crucial for training the LLM.\n\n2. Model Architecture: This involves selecting and designing the architecture of the LLM, such as the type of neural network and the number of parameters.\n\n3. Training the Model at Scale: This involves training the LLM on the prepared dataset, which requires significant computational resources and techniques to optimize the training process.\n\n4. Evaluating the Model: This involves assessing the performance of the trained LLM on various tasks and benchmarks to determine its effectiveness and limitations.'}
Reply:  The four steps that can be used to build LLM from scratch are:

1. Data Curation: This involves gathering and preparing a large and diverse dataset of text, which is crucial for training the LLM.

2. Model Architecture: This involves selecting and designing t